<a href="https://colab.research.google.com/github/Harshavardhan18/Dog_Breed_Classifier/blob/master/Dog_Breed_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DOG BREED CLASSIFIER**

---





Run the below cell to import all necessary library required for building the model.

In [0]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
from glob import glob
import tensorflow as tf
from keras.preprocessing import image    
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from PIL import ImageFile    
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras import optimizers
import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline   

Run the below cell **only if** your are using google drive to store your dataset.

The below code statement is responsible for connecting to your google drive.



In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Function to load train, test, and validation datasets and print the stats about the dataset.

**TODO:**


1.   Change the number of class labels in load_dataset().

> Example: If you want to predict for only four breed, then change number of classes to 4.


2.   Provide path for training, validation and test dataset.(If in case you are using google drive then provide the path accordingly)


3.   **Only edit** the content enclosed within ****





In [0]:
# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), #change to number of classification label )
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('**PATH TO TRAINING DATASET**')
valid_files, valid_targets = load_dataset('**PATH TO VALIDATION DATASET**')
test_files, test_targets = load_dataset('**PATH TO TESTING DATASET**')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("**PATH TO TRAINING DATASET**/*/"))]

# print statistics about the dataset
print(dog_names)
print('There are %d total dog categories.' % len(dog_names))
print('There are %d total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

Function that converts an image to numpy array with target size of 224x224x3 and finally returns a stack of list of images that have been converted as numpy array.

**Option**:
        You many wish to change the target size of the image.





In [0]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

Call's the above fuction with the train, test, and validation list and normalize the data in the returned array.

In [0]:
# pre-process the data (Normalize)

ImageFile.LOAD_TRUNCATED_IMAGES = True                 
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

Perform image augmentation using **ImageDataGenerator** 

**Option:**
You may choose to change the arguments such as flipping, shifs by changing range to 15% or 20% or any of your choice.


In [0]:
# Image Augmentation
# create and configure augmented image generator
datagen = ImageDataGenerator(
                    width_shift_range=0.1,   
                    height_shift_range=0.1,  
                    horizontal_flip=True) 

# fit augmented image generator on data
datagen.fit(train_tensors)

Define a model using Sequental() in keras consisting of 

1.   Input layer as Batch Normalization
2.   6 layers of Convolution, Pooling and Batch normalization as regulizer and activation function as Relu (Rectified Linear Unit)
3.   Flatten the layer.
4.   5 hidden ouput layer each having Relu as its activation function
5.   An output layer consisting of number of target labels and activation function as softmax which gives propability distribution value over the list of target classes.



**TODO:**

1.  If in case you have added more or less number of dog breed to train, then change number of nodes in the last layer to number of target classes. 


**Options:** You may choose to experiment the model by 

1.   Adding or removing the CNN layers.
2.   Remove Batch Normalization.
3.   Change the filter size, kernal size or activation function to sigmoid, tanh to see the results.
4.   Modify pool size it is usually 1 or 2.
5.   Add or remove dense layer.
6.   Change the activation function in dense layer.
7.   Change the value in dropout, usually inbetween 0.25 to 0.5 or as much as you prefer for experimenting purpose.










In [0]:
#Define a CNN Model
model = Sequential()

model.add(BatchNormalization(input_shape=(224, 224, 3)))
model.add(Conv2D(filters=16, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=32, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=64, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=128, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=256, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=512, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(32,activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(16,activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(5, activation='softmax'))

model.summary()

Compile the model using **adam** optimizer.

1.   Learning rate of 0.001.
2.   Loss function as categorical cross entropy.
3.   metrics which defines accuracy of the model.

**Option:** Feel free to change the learning rate or loss functions any of your choice.




In [0]:
# Compile the model

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

Evaluate the model before training. The model returns two values:
1.   Initial loss value.
2.   Initial metrics value which gives how good the model performs.




In [0]:
#Evaluate The model
model.evaluate(x=valid_tensors, y=valid_targets, batch_size=None, verbose=1, sample_weight=None, steps=None)

Train the model by choosing the parametes as:

1.   epochs = 50
2.   batch size = 128

**TODO:**
1.   Provide the path for storing the check point **.hdf5** format (If in case you are using google drive then provide the path accordingly)
2.   **Only edit** the content enclosed within ****

**Option:** 
You may choose to experiment by changing
1.   Number of epochs
2.   Batch size





In [0]:
#Train the model
epochs = 50

batch_size = 128

checkpointer = ModelCheckpoint(filepath='**PROVIDE A PATH TO STORE THE WEIGHT FILE IN THE DRIVE IN .hdf5 FORMAT**', 
                               verbose=1, save_best_only=True)

# Using Image Augmentation
model.fit_generator(datagen.flow(train_tensors, train_targets, batch_size=batch_size),
                    validation_data=(valid_tensors, valid_targets), 
                    steps_per_epoch=train_tensors.shape[0] // batch_size,
                    epochs=epochs, callbacks=[checkpointer], verbose=1)

Evaluate the model after training. The model returns two values:
1.   Loss value.
2.   Metrics value which gives how good the model performs.

In [0]:
model.evaluate(x=valid_tensors, y=valid_targets, batch_size=None, verbose=1, sample_weight=None, steps=None)

Gets index of predicted dog breed for each image in test set and prints the accuracy of the model on the test dataset.

In [0]:
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Run the cell if you want to save your model and the weight.

**TODO:**
1.   Provide the path where the model and the weight file is to be stored(If in case you are using google drive then provide the path accordingly)
2.       **Only edit** the content enclosed within ****








In [0]:
model.save("**PATH WHERE THE MODEL IS TO BE STORED IN .h5 FORMAT**")
model.save_weights("**PATH WHERE THE WEIGHT FILE IS TO BE STORED IN .hdf5 FORMAT**")

Load the model and its corresponding weight file that is stored in the path.

**TODO:**
1.   Provide the path where the model and the weight file is stored(If in case you are using google drive then provide the path accordingly)
2.       **Only edit** the content enclosed within ****
3.       Modify the** dog_breed** variable to include the breed names for which you have trained your model to classify.





In [0]:
dog_breed = ['Affenpinscher','Afghan_hound','Australian_shepherd','Bichon_frise','Golden_retriever']

model = load_model('**PATH WHERE THE MODEL IS STORED IN .h5 FORMAT**')
model.load_weights('**PATH WHERE THE WEIGHT FILE IS STORED IN .hdf5 FORMAT**')
model.summary()

def predict_breed(np_arr):
    return np.argmax(model.predict(np.expand_dims(np_arr, axis=0)))

def dog_breed_classifier(np_arr, img_path, output):
    find_breed = predict_breed(np_arr)
    img = cv2.imread(img_path)
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    print("The correct dog breed is " + dog_breed[np.argmax(output) - 1])
    print("The detected dog breed is " + dog_breed[find_breed - 1])
    plt.imshow(cv_rgb)
    plt.show()
    
for img, nparr, output in zip(test_files, test_tensors, test_targets):
    dog_breed_classifier(nparr, img, output)